In [1]:
#对一些app如9,19,10等，hour 6, 11, 15有问题，可以考虑单独对19建模
#感觉时间序列分析是关键，双11行为变化难以预测

In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm
from sklearn import model_selection

In [2]:
train_file_in = '../data/train.csv'
sub_file_in = '../data/test.csv'

In [ ]:
#df = pd.read_csv('filename.tar.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [3]:
dtypes = {
'ip'            : 'uint32',
'app'           : 'uint16',
'device'        : 'uint16',
'os'            : 'uint16',
'channel'       : 'uint16',
'is_attributed' : 'uint8',
'click_id'      : 'uint32'
}
df = pd.read_csv(train_file_in, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'], dtype=dtypes)
df_sub = pd.read_csv(sub_file_in, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'], dtype=dtypes)

In [4]:
len_train = df.shape[0]
df = df.append(df_sub)

In [5]:
print('add feature hour, day')
df['hour'] = pd.to_datetime(df.click_time).dt.hour.astype('uint8')
df['day'] = pd.to_datetime(df.click_time).dt.day.astype('uint8')

add feature hour, day


In [6]:
print('grouping by ip-day-hour, count channel')
gp = df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_day_hour_count'})
print("merging...")
df = df.merge(gp, on=['ip','day','hour'], how='left')
del gp
gc.collect()

grouping by ip-day-hour, count channel
merging...


107

In [7]:
print('group by ip-app, count channel')
gp = df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
print("merging...")
df = df.merge(gp, on=['ip','app'], how='left')
del gp
gc.collect()

group by ip-app, count channel
merging...


66

In [8]:
print('group by ip-app-os, count channel')
gp = df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
print("merging...")
df = df.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

group by ip-app-os, count channel
merging...


117

In [9]:
import pickle

In [10]:
len_train

184903890

In [11]:
with open('../data/features.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)